In [1]:
from google.cloud import storage
import pandas as pd

bucket_name = "nobu_w266_final_project"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

# When you have your files in a subfolder of the bucket.
my_prefix = "OECD_file/" # the name of the subfolder
blobs = bucket.list_blobs(prefix = my_prefix, delimiter = '/')

for blob in blobs:
    if(blob.name != my_prefix): # ignoring the subfolder itself 
        file_name = blob.name.replace(my_prefix, "")
        blob.download_to_filename(file_name) # download the file to the machine
        df = pd.read_csv(file_name, delimiter="|") # load the data
        print(df)

          appln_id        app_nbr  filing  tech_field  many_field  \
0         16427085  EP19780100001    1978        16.0           1   
1         16427086  EP19780100002    1978        31.0           0   
2         16427087  EP19780100003    1978        32.0           1   
3         16427088  EP19780100004    1978        14.0           1   
4         16427089  EP19780100005    1978         8.0           0   
...            ...            ...     ...         ...         ...   
3507894  506385435  EP20190405001    2019         8.0           1   
3507895  506385437  EP20190405002    2019        24.0           0   
3507896  507537560  EP20190460001    2019        35.0           0   
3507897  505638106  EP20190461501    2019         1.0           1   
3507898  506424754  EP20190700348    2019         6.0           0   

         patent_scope  family_size  grant_lag  bwd_cits  npl_cits  ...  \
0                   3            7     1203.0         4         0  ...   
1                   1  

In [2]:
# Now try to pull the patent data combining 2 files to accumulate 10 years
from google.cloud import storage
import pandas as pd

bucket_name = "nobu_w266_final_project"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

# When you have your files in a subfolder of the bucket.
my_prefix = "Patent_EP/" # the name of the subfolder
blobs = bucket.list_blobs(prefix = my_prefix, delimiter = '/')

patent_data_combine = None

for blob in blobs:
    if(blob.name != my_prefix): # ignoring the subfolder itself 
        file_name = blob.name.replace(my_prefix, "")
        blob.download_to_filename(file_name) # download the file to the machine
        patent_data = pd.read_csv(file_name) # load the data
        # print(patent_data)
        if patent_data_combine is None:
            patent_data_combine = patent_data
        else:
            patent_data_combine = patent_data_combine.append(patent_data)

In [3]:
patent_data_combine.sort_values('filing_date')

,publication_number,application_number,text,filing_date
161697,EP-2393505-A1,EP-10705790-A,The present invention relates to the use of a ...,20100102
11332,EP-2376972-A1,EP-10708886-A,"A parallel-sighted, image-reversing prism syst...",20100102
52710,EP-2384189-A2,EP-10726819-A,Improved compositions for tissue augmentation ...,20100103
74016,EP-2374013-A2,EP-10708389-A,Apparatus for radiation mapping of a region co...,20100103
157739,EP-2374128-A2,EP-10703964-A,A device for recording at least part of a data...,20100103
...,...,...,...,...
16172,EP-3572775-A2,EP-19193474-A,A switch control circuit and method for an ele...,20190823
19388,EP-3569717-A2,EP-19194106-A,The present invention is directed to a diagnos...,20190828
19389,EP-3569717-A2,EP-19194106-A,The present invention is directed to a diagnos...,20190828
12887,EP-3564854-A1,EP-19196510-A,Embodiments of the present invention provide a...,20190910


In [4]:
patent_data_combine['new_appl_nbr'] = 'EP20' + patent_data_combine['application_number'].str[3:5] + '0' + patent_data_combine['application_number'].str[5:-2]

In [5]:
patent_data_combine

,publication_number,application_number,text,filing_date,new_appl_nbr
0,EP-2493772-A1,EP-10773469-A,This invention pertains to a U-board split cas...,20101020,EP20100773469
1,EP-2496032-A1,EP-10826481-A,To transmit simultaneously a reference signal ...,20101004,EP20100826481
2,EP-2610125-A1,EP-10856430-A,A vehicle control device includes a diesel eng...,20100826,EP20100856430
3,EP-2436619-A2,EP-11190626-A,"A vertical conveyor system (150A, 150B) for tr...",20100412,EP20110190626
4,EP-2813266-A2,EP-14172995-A,A fire suppression system (20) includes a cont...,20100825,EP20140172995
...,...,...,...,...,...
19963,EP-3517373-A1,EP-19152079-A,A seatbelt pretensioning retractor assembly fo...,20190116,EP20190152079
19964,EP-3566652-A1,EP-19172131-A,A system and method for constructing virtual r...,20190501,EP20190172131
19965,EP-3566652-A1,EP-19172131-A,A system and method for constructing virtual r...,20190501,EP20190172131
19966,EP-3545862-A2,EP-19165960-A,A method of compressing tissue during a surgic...,20190328,EP20190165960


In [6]:
patent_data_combine.drop_duplicates(subset='new_appl_nbr', keep = 'first', inplace = True)
patent_data_combine

,publication_number,application_number,text,filing_date,new_appl_nbr
0,EP-2493772-A1,EP-10773469-A,This invention pertains to a U-board split cas...,20101020,EP20100773469
1,EP-2496032-A1,EP-10826481-A,To transmit simultaneously a reference signal ...,20101004,EP20100826481
2,EP-2610125-A1,EP-10856430-A,A vehicle control device includes a diesel eng...,20100826,EP20100856430
3,EP-2436619-A2,EP-11190626-A,"A vertical conveyor system (150A, 150B) for tr...",20100412,EP20110190626
4,EP-2813266-A2,EP-14172995-A,A fire suppression system (20) includes a cont...,20100825,EP20140172995
...,...,...,...,...,...
19957,EP-3567539-A1,EP-19171027-A,Distributing resources in a predetermined geog...,20190425,EP20190171027
19960,EP-3527317-A1,EP-19156479-A,Embodiments of hybrid engine welding systems a...,20190211,EP20190156479
19962,EP-3517373-A1,EP-19152079-A,A seatbelt pretensioning retractor assembly fo...,20190116,EP20190152079
19964,EP-3566652-A1,EP-19172131-A,A system and method for constructing virtual r...,20190501,EP20190172131


In [7]:
df_merge = pd.merge(patent_data_combine, df, left_on='new_appl_nbr', right_on='app_nbr', how='inner')

In [8]:
df_merge

,publication_number,application_number,text,filing_date,new_appl_nbr,appln_id,app_nbr,filing,tech_field,many_field,...,fwd_cits7,fwd_cits7_xy,breakthrough,breakthrough_xy,generality,originality,radicalness,renewal,quality_index_4,quality_index_6
0,EP-2493772-A1,EP-10773469-A,This invention pertains to a U-board split cas...,20101020,EP20100773469,328773105,EP20100773469,2010,25.0,0,...,0,0,NaN,NaN,NaN,0.602041,0.000000,4.0,0.396296,0.393637
1,EP-2496032-A1,EP-10826481-A,To transmit simultaneously a reference signal ...,20101004,EP20100826481,333779641,EP20100826481,2010,4.0,1,...,0,0,NaN,NaN,NaN,0.839506,0.111111,NaN,NaN,NaN
2,EP-2610125-A1,EP-10856430-A,A vehicle control device includes a diesel eng...,20100826,EP20100856430,341900782,EP20100856430,2010,32.0,0,...,0,0,NaN,NaN,NaN,0.867139,0.224490,NaN,NaN,NaN
3,EP-2436619-A2,EP-11190626-A,"A vertical conveyor system (150A, 150B) for tr...",20100412,EP20110190626,339215521,EP20110190626,2010,25.0,0,...,9,4,NaN,NaN,0.180000,0.764543,0.157895,NaN,NaN,NaN
4,EP-2813266-A2,EP-14172995-A,A fire suppression system (20) includes a cont...,20100825,EP20140172995,419197262,EP20140172995,2010,24.0,0,...,6,2,NaN,NaN,0.104938,0.887755,0.571429,9.0,0.420142,0.298506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130300,EP-3512074-A1,EP-19150413-A,The invention relates to a permanent magnet ro...,20190104,EP20190150413,505250492,EP20190150413,2019,1.0,0,...,0,0,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN
1130301,EP-3514984-A1,EP-19152912-A,The invention provides a method for determinin...,20190121,EP20190152912,506365616,EP20190152912,2019,3.0,0,...,0,0,NaN,NaN,NaN,0.908163,0.857143,NaN,NaN,NaN
1130302,EP-3514738-A1,EP-19152968-A,A demand prediction apparatus configured to pr...,20190122,EP20190152968,506385155,EP20190152968,2019,7.0,0,...,0,0,NaN,NaN,NaN,0.000000,1.000000,NaN,NaN,NaN
1130303,EP-3514451-A1,EP-19151645-A,The present invention relates generally to a f...,20190114,EP20190151645,505895332,EP20190151645,2019,1.0,0,...,0,0,NaN,NaN,NaN,0.843750,0.375000,NaN,NaN,NaN


In [9]:
df_merge_quality = df_merge[(df_merge['quality_index_6'] >=0) & (len(df_merge['text']) >0)]
df_merge_quality

,publication_number,application_number,text,filing_date,new_appl_nbr,appln_id,app_nbr,filing,tech_field,many_field,...,fwd_cits7,fwd_cits7_xy,breakthrough,breakthrough_xy,generality,originality,radicalness,renewal,quality_index_4,quality_index_6
0,EP-2493772-A1,EP-10773469-A,This invention pertains to a U-board split cas...,20101020,EP20100773469,328773105,EP20100773469,2010,25.0,0,...,0,0,NaN,NaN,NaN,0.602041,0.000000,4.0,0.396296,0.393637
4,EP-2813266-A2,EP-14172995-A,A fire suppression system (20) includes a cont...,20100825,EP20140172995,419197262,EP20140172995,2010,24.0,0,...,6,2,NaN,NaN,0.104938,0.887755,0.571429,9.0,0.420142,0.298506
5,EP-2255683-B1,EP-10163204-A,The device has an application device (47) for ...,20100519,EP20100163204,315607513,EP20100163204,2010,34.0,0,...,1,1,NaN,NaN,NaN,0.642361,0.416667,9.0,0.278788,0.367996
6,EP-2262981-B1,EP-10711585-A,The invention relates to a bimetallic valve fo...,20100329,EP20100711585,298884917,EP20100711585,2010,27.0,0,...,0,0,NaN,NaN,NaN,0.842951,0.675676,7.0,0.300938,0.450909
7,EP-2462850-B1,EP-10194739-A,The present invention concerns a beverage prep...,20101213,EP20100194739,329540694,EP20100194739,2010,33.0,0,...,1,1,NaN,NaN,0.000000,0.668639,0.461538,9.0,0.273246,0.367701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129085,EP-3346246-A1,EP-18150116-A,Systems and methods for icing resistant total ...,20180102,EP20180150116,488117740,EP20180150116,2018,10.0,0,...,0,0,NaN,NaN,NaN,0.720000,0.600000,1.0,0.281746,0.206609
1129096,EP-3407148-A1,EP-18156651-A,Provided is a highly convenient communication ...,20180214,EP20180156651,489969681,EP20180156651,2018,12.0,1,...,0,0,NaN,NaN,NaN,0.710744,0.363636,1.0,0.357143,0.267411
1129125,EP-3388275-A1,EP-18165679-A,A control device of a four-wheel-drive vehicle...,20180404,EP20180165679,492667799,EP20180165679,2018,31.0,1,...,0,0,NaN,NaN,NaN,0.856099,0.378378,1.0,0.277778,0.292488
1129149,EP-3372398-A1,EP-18159484-A,Roller blind of a sheet material having a sand...,20180301,EP20180159484,490863830,EP20180159484,2018,21.0,0,...,0,0,NaN,NaN,NaN,0.000000,1.000000,1.0,0.274074,0.180979


In [10]:
#
#conditions = [
#    (df_merge_quality['quality_index_6'] >= 0) & (df_merge_quality['quality_index_6'] < 0.15),
#    (df_merge_quality['quality_index_6'] >= 0.15) & (df_merge_quality['quality_index_6'] < 0.3),
#    (df_merge_quality['quality_index_6'] >= 0.3) & (df_merge_quality['quality_index_6'] < 0.45),
#    (df_merge_quality['quality_index_6'] >= 0.45) & (df_merge_quality['quality_index_6'] < 0.6),
#    (df_merge_quality['quality_index_6'] >= 0.6)
#]

#choices = ['0', '1', '2', '3', '4']
#**//

In [10]:
conditions = [

    (df_merge_quality['quality_index_6'] < 0.3),
    (df_merge_quality['quality_index_6'] >= 0.3)
]

choices = ['0', '1']

In [11]:
import numpy as np
df_merge_quality['quality_rank'] = np.select(condlist=conditions, choicelist=choices)

/home/nobu_yamaguchi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
def tech_field_cat(subcat):
    if subcat <= 8:
        return '0' # for 1. Electrical engineering
    elif subcat <= 13:
        return '1' # for 2. Instruments
    elif subcat <= 24:
        return '2' # for 3. Chemistry
    elif subcat <= 32:
        return '3' # for 4. Mechanical Engineering
    else:
        return '4' # for 5. Other fields

In [14]:
df_merge_quality['tech_field_big_cat'] = df_merge_quality['tech_field'].apply(tech_field_cat)

/home/nobu_yamaguchi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
df_merge_quality['quality_rank'].value_counts()

0    236469
1    164572
Name: quality_rank, dtype: int64

In [16]:
import numpy as np
import pandas as pd
import pickle as cPickle
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

Using TensorFlow backend.


In [17]:
MAX_SENT_LENGTH = 30
MAX_SENTS = 3
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [18]:
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

In [19]:
df_margin_quality_filter = df_merge_quality[df_merge_quality['tech_field_big_cat'] == '1']
data_train = df_margin_quality_filter.sample(frac=1)[:5000]
data_train

,publication_number,application_number,text,filing_date,new_appl_nbr,appln_id,app_nbr,filing,tech_field,many_field,...,breakthrough,breakthrough_xy,generality,originality,radicalness,renewal,quality_index_4,quality_index_6,quality_rank,tech_field_big_cat
686126,EP-3089719-A1,EP-14827966-A,A conformable wound dressing system is provide...,20141222,EP20140827966,425817907,EP20140827966,2014,13.0,0,...,NaN,NaN,NaN,0.864051,0.678161,4.0,0.274074,0.355498,1,1
501049,EP-2914938-A1,EP-13785470-A,The invention relates to an air mass flowmeter...,20131101,EP20130785470,412260124,EP20130785470,2013,10.0,0,...,NaN,NaN,NaN,0.809600,0.560000,5.0,0.205128,0.266640,0,1
826317,EP-3209245-A1,EP-15801232-A,The present invention concerns an elbow prosth...,20150930,EP20150801232,447044998,EP20150801232,2015,13.0,0,...,NaN,NaN,NaN,0.000000,0.000000,4.0,0.487179,0.363289,1,1
239420,EP-2539015-A2,EP-11748237-A,The invention provides a retinal prosthetic me...,20110228,EP20110748237,336616892,EP20110748237,2011,13.0,1,...,NaN,NaN,0.0,0.873110,0.692308,8.0,0.331100,0.329578,1,1
354340,EP-2753390-A1,EP-12762054-A,Methods and system of respiratory therapy trea...,20120731,EP20120762054,375729664,EP20120762054,2012,13.0,0,...,NaN,NaN,NaN,0.448980,0.142857,6.0,0.135338,0.186410,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225040,EP-2436356-A3,EP-11183664-A,The invention relates to a closing insert (301...,20111003,EP20110183664,337364653,EP20110183664,2011,13.0,0,...,NaN,NaN,NaN,0.375000,0.250000,7.0,0.150327,0.195563,0,1
192540,EP-2786165-A1,EP-11793393-A,The document relates to a method for the deter...,20111130,EP20110793393,339968430,EP20110793393,2011,10.0,0,...,NaN,NaN,NaN,0.625000,0.000000,7.0,0.168803,0.218604,0,1
822851,EP-3030862-A1,EP-15778200-A,Disclosed is a circuit arrangement (1) for con...,20150707,EP20150778200,445447042,EP20150778200,2015,10.0,0,...,NaN,NaN,NaN,0.866782,0.705882,3.0,0.257576,0.387824,1,1
396667,EP-2780694-A1,EP-12788175-A,For tracking a movement of a particle (2) in a...,20121112,EP20120788175,378575824,EP20120788175,2012,10.0,0,...,NaN,NaN,NaN,0.851211,0.529412,6.0,0.386752,0.378634,1,1


In [20]:
data_train.to_csv('patent_abstract_5000_filter.csv')

In [215]:
import nltk
from nltk import tokenize

abstracts = [] # abstracts is list of list of list to hold each sentences of each abstract (the most complete data)
labels = [] # label is just a list holding our label which is quality_index
texts = []  # texts to hold each complete abstract as list of list (note: abstract not breaking up to sentence level)
for idx in range(data_train.text.shape[0]): # for each row
    text = clean_str(str(data_train.iloc[idx]['text'])) # text is each complete abstract
    texts.append(text) # texts to hold each complete abstract as list of string (note: abstract not breaking up to sentence level)
    sentences = tokenize.sent_tokenize(text) # sentences is list of string holding each complete sentence 
    abstracts.append(sentences) # abstracts is list of list of string to hold each sentences of each abstract (the most complete data)
    labels.append(data_train.iloc[idx]['quality_rank']) # label is just a list holding our label which is quality_index

In [216]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS) # intend to use next line .fit_on_texts to index each word within specific abstract at current iteration/loop, the more frequent word has lower index number, it is a dictionary format, it's like a unique vocabulary index
tokenizer.fit_on_texts(texts)
data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

In [217]:
tokenizer.word_counts

OrderedDict([('the', 51051),
             ('invention', 2808),
             ('is', 9909),
             ('based', 688),
             ('on', 3317),
             ('a', 37611),
             ('contactless', 20),
             ('current', 561),
             ('measuring', 266),
             ('device', 3224),
             ('10', 1653),
             ('for', 7335),
             ('determining', 366),
             ('consumption', 62),
             ('energy', 329),
             ('of', 22437),
             ('consumer', 19),
             ('14', 732),
             ('which', 3922),
             ('can', 1408),
             ('be', 2334),
             ('connected', 1023),
             ('by', 3761),
             ('means', 1293),
             ('plug', 120),
             ('32', 297),
             ('to', 15865),
             ('an', 8200),
             ('electrical', 447),
             ('ac', 70),
             ('or', 4646),
             ('three', 187),
             ('phase', 362),
             ('power', 949),
 

In [218]:
tokenizer.document_count

5000

In [219]:
len(tokenizer.word_index) # 143937 unqiue vocabulary out of the entire abstract dataset

17536

In [220]:
for i, sentences in enumerate(abstracts):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
            #for word in wordTokens:
                if word in tokenizer.word_index.keys():
                    if (k < MAX_SENT_LENGTH) and (tokenizer.word_index[word] < MAX_NB_WORDS):
                        data[i, j, k] = tokenizer.word_index[word]
                        k = k + 1

In [221]:
word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

Total 17536 unique tokens.


In [222]:
np.unique(np.asarray(labels))

array(['0', '1'], dtype='<U1')

In [223]:
labels = to_categorical(np.asarray(labels), num_classes=2) 
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (5000, 3, 30)
Shape of label tensor: (5000, 2)


In [224]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [225]:
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Number of positive and negative reviews in traing and validation set
[2367. 1633.]
[604. 396.]


In [226]:
x_train[2]

array([[    8,    99,    42,     2,  3453,  1368,  6354,   311,    22,
           14,  1046,     2,   101,     9,   311,     1,  3453,  6354,
            5,    95,     1,   101,    49,    27,     8,   848,   154,
          101,  1247,     8],
       [   87,    46,     7,     2,    29,     9,  1382, 16297,   309,
           14,    42,     2,   220,     3,   118,     1,   101,     9,
          311,     1,  3453,  1368,  6354,     5,    95,     0,     0,
            0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0]], dtype=int32)

In [227]:
x_val[10]

array([[  189,    24,   590,     4,   198,     9,   624,  3546,   598,
         6379,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [    1,    46,   198,   274,     1,   537,     3,  1540,    11,
        10799, 10800,   598,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [   87,    46,    24,   590,     4,   198,     3,   118,  1540,
           11, 10799, 10800,   598,     9,     1,   309,     3,  3546,
          598,  6379,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0]], dtype=int32)

In [228]:
x_train.shape

(4000, 3, 30)

In [229]:
# Standard python helper libraries.
import os, sys, re, json, time
import itertools, collections
from importlib import reload
from IPython.display import display

# NumPy and SciPy for matrix ops

import scipy.sparse


# Helper libraries
from w266_common import utils, vocabulary, tf_embed_viz
import glove_helper; reload(glove_helper)

<module 'glove_helper' from '/home/nobu_yamaguchi/final_project/glove_helper.py'>

In [88]:
GLOVE_DIR = "data/glove"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [230]:
print('Total %s word vectors.' % len(embeddings_index))

Total 400000 word vectors.


In [231]:

embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
print(embedding_matrix.shape)
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

(17537, 100)


In [232]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LENGTH,
                            trainable=False)

sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
sentEncoder = Model(sentence_input, l_lstm)

review_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(LSTM(100))(review_encoder)
l_lstm_dense = Dense(100, activation='relu')(l_lstm_sent)
l_lstm_sent_drop = Dropout(rate=0.5)(l_lstm_dense)
preds = Dense(2, activation='relu')(l_lstm_sent_drop) # [Steven] why 5 will throw error and require 6 ? it is weird since we only have 5 number scale
model = Model(review_input, preds)

In [206]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [62]:
print("model fitting - Hierachical LSTM")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10, batch_size=50)

model fitting - Hierachical LSTM


/home/nobu_yamaguchi/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4000 samples, validate on 1000 samples
Epoch 1/10
4000/4000 [==============================] - 31s 8ms/step - loss: 0.6824 - acc: 0.5865 - val_loss: 0.6771 - val_acc: 0.5880
Epoch 2/10
4000/4000 [==============================] - 29s 7ms/step - loss: 0.6762 - acc: 0.5888 - val_loss: 0.6803 - val_acc: 0.5950
Epoch 3/10
4000/4000 [==============================] - 29s 7ms/step - loss: 0.6745 - acc: 0.5872 - val_loss: 0.6714 - val_acc: 0.5880
Epoch 4/10
4000/4000 [==============================] - 29s 7ms/step - loss: 0.6609 - acc: 0.5995 - val_loss: 0.6823 - val_acc: 0.5930
Epoch 5/10
4000/4000 [==============================] - 29s 7ms/step - loss: 0.6428 - acc: 0.6288 - val_loss: 0.6817 - val_acc: 0.5740
Epoch 6/10
4000/4000 [==============================] - 29s 7ms/step - loss: 0.5999 - acc: 0.6790 - val_loss: 0.7662 - val_acc: 0.5020
Epoch 7/10
4000/4000 [==============================] - 29s 7ms/step - loss: 0.5403 - acc: 0.7262 - val_loss: 0.7414 - val_acc: 0.5820
Epoch 8

In [233]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [234]:
print("model fitting - Hierachical LSTM")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10, batch_size=50)

model fitting - Hierachical LSTM
Train on 4000 samples, validate on 1000 samples
Epoch 1/10
4000/4000 [==============================] - 21s 5ms/step - loss: 0.9263 - acc: 0.5307 - val_loss: 0.6751 - val_acc: 0.5860
Epoch 2/10
4000/4000 [==============================] - 18s 5ms/step - loss: 0.7100 - acc: 0.5456 - val_loss: 0.6742 - val_acc: 0.6035
Epoch 3/10
4000/4000 [==============================] - 18s 5ms/step - loss: 0.7043 - acc: 0.5455 - val_loss: 0.6697 - val_acc: 0.6020
Epoch 4/10
4000/4000 [==============================] - 18s 5ms/step - loss: 0.6959 - acc: 0.5556 - val_loss: 0.6750 - val_acc: 0.5885
Epoch 5/10
4000/4000 [==============================] - 19s 5ms/step - loss: 0.6942 - acc: 0.5570 - val_loss: 0.6688 - val_acc: 0.6035
Epoch 6/10
4000/4000 [==============================] - 18s 5ms/step - loss: 0.6947 - acc: 0.5616 - val_loss: 0.6634 - val_acc: 0.6055
Epoch 7/10
4000/4000 [==============================] - 18s 5ms/step - loss: 0.7671 - acc: 0.5260 - val_loss: